In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))  # يضيف customer_journey إلى PYTHONPATH
from src.system import CustomerJourneySystem


In [2]:
import pandas as pd

df = pd.read_csv("../data/processed/cleaned_activities.csv")
df["types"] = df["types"].fillna("Unknown").astype(str).str.strip()

df = df.sort_values(["account_id", "activity_order"])
df["next_action"] = df.groupby("account_id")["types"].shift(-1)

model_df = df[df["next_action"].notna()].copy()
model_df.head()


,account_id,SourceSystem,activity_date,who_id,opportunity_id,opportunity_stage,is_lead,types,Country,solution,activity_order,action_weight,next_action
0,0010L00001hVmFhQAK,SFDC_US,2022-07-25,0030L00001vlbHLQAY,NaN,no_opp,1,Follow Up,US,MRS,1.0,1.0,Call
1,0010L00001hVmFhQAK,SFDC_US,2023-02-08,0034X00002xZlQtQAK,NaN,no_opp,1,Call,US,MRS,2.0,1.0,Call
2,0010L00001hVmFhQAK,SFDC_US,2023-02-14,0030L00001vlbHLQAY,NaN,no_opp,1,Call,US,MRS,3.0,1.0,Call
3,0010L00001hVmFhQAK,SFDC_US,2023-02-20,0030L00001vlbHLQAY,NaN,no_opp,2,Call,US,MRS,4.0,1.0,Call
4,0010L00001hVmFhQAK,SFDC_US,2023-03-16,0034X00003GOUrFQAX,NaN,no_opp,1,Call,US,MRS,5.0,1.0,Call


In [3]:
top4_by_country = (
    model_df.groupby(["Country", "next_action"])
            .size()
            .reset_index(name="count")
            .sort_values(["Country", "count"], ascending=[True, False])
            .groupby("Country")
            .head(4)
            .reset_index(drop=True)
)


top4_by_country.to_csv("../data/processed/top4_next_actions_by_country.csv", index=False)
top4_by_country.head(20)

,Country,next_action,count
0,AT,Email,204
1,AT,Follow Up,2
2,AT,1St Appointment,1
3,AT,2Nd Appointment,1
4,BE,Email,358
5,BE,1St Appointment,28
6,BE,2Nd Appointment,25
7,BE,Meeting,14
8,CA,Email,444
9,CA,Follow Up,132


In [4]:
top4_by_solution = (
    model_df.groupby(["solution", "next_action"])
            .size()
            .reset_index(name="count")
            .sort_values(["solution", "count"], ascending=[True, False])
            .groupby("solution")
            .head(4)
            .reset_index(drop=True)
)

top4_by_solution.head(20)
top4_by_solution.to_csv("../data/processed/top4_next_actions_by_solution.csv", index=False)
top4_by_solution.head(20)

,solution,next_action,count
0,Digital,Email,3351
1,Digital,Call,2889
2,Digital,Meeting,782
3,Digital,Follow Up,690
4,MRS,Email,68457
5,MRS,Call,42856
6,MRS,Meeting,15846
7,MRS,Follow Up,11593
8,PLS,Email,498
9,PLS,Call,188


In [5]:
top4_by_country_solution = (
    model_df.groupby(["Country", "solution", "next_action"])
            .size()
            .reset_index(name="count")
            .sort_values(["Country", "solution", "count"], ascending=[True, True, False])
            .groupby(["Country", "solution"])
            .head(4)
            .reset_index(drop=True)
)
top4_by_country_solution.to_csv("../data/processed/top4_next_actions_by_country_solution.csv", index=False)

top4_by_country_solution.head(30)


,Country,solution,next_action,count
0,AT,MRS,Email,204
1,AT,MRS,Follow Up,2
2,AT,MRS,1St Appointment,1
3,AT,MRS,2Nd Appointment,1
4,BE,Digital,Email,8
5,BE,Digital,Follow Up,2
6,BE,MRS,Email,350
7,BE,MRS,1St Appointment,28
8,BE,MRS,2Nd Appointment,25
9,BE,MRS,Meeting,14


In [7]:
from src.system import CustomerJourneySystem

sys = CustomerJourneySystem(processed_dir="../data/processed")

# تجريب أي country/solution (: AT/MRS أو BE/MRS أو BE/Digital)
out = sys.add_account(account_id="0010L00001hVmFnQAK", country="AT", solution="MRS")
out


{'account_id': '0010L00001hVmFnQAK',
 'country': 'AT',
 'solution': 'MRS',
 'top4_by_country': [{'Country': 'AT', 'next_action': 'Email', 'count': 204},
  {'Country': 'AT', 'next_action': 'Follow Up', 'count': 2},
  {'Country': 'AT', 'next_action': '1St Appointment', 'count': 1},
  {'Country': 'AT', 'next_action': '2Nd Appointment', 'count': 1}],
 'top4_by_solution': [{'solution': 'MRS',
   'next_action': 'Email',
   'count': 68457},
  {'solution': 'MRS', 'next_action': 'Call', 'count': 42856},
  {'solution': 'MRS', 'next_action': 'Meeting', 'count': 15846},
  {'solution': 'MRS', 'next_action': 'Follow Up', 'count': 11593}],
 'top4_by_country_solution': [{'Country': 'AT',
   'solution': 'MRS',
   'next_action': 'Email',
   'count': 204},
  {'Country': 'AT', 'solution': 'MRS', 'next_action': 'Follow Up', 'count': 2},
  {'Country': 'AT',
   'solution': 'MRS',
   'next_action': '1St Appointment',
   'count': 1},
  {'Country': 'AT',
   'solution': 'MRS',
   'next_action': '2Nd Appointment'

In [9]:

# 1) عرض Top4 baseline (بدون أوزان حساب)
acc_out = sys_obj.add_account("A001", "AT", "MRS")
print("Baseline CS:", acc_out["top4_by_country_solution"])

# 2) إضافة Actions (تحديث state + rerank)
a1 = sys_obj.add_action("A001", "AT", "MRS", "Email")
print("After Email:", a1["adjusted_weight"])
print(a1["top4_by_country_solution"])

a2 = sys_obj.add_action("A001", "AT", "MRS", "Follow Up")
print("After Follow Up:", a2["adjusted_weight"])
print(a2["top4_by_country_solution"])

# 3) إثبات حفظ الحالة
state = json.loads(Path("../data/processed/account_state.json").read_text(encoding="utf-8"))
print("touch_count:", state["A001"]["touch_count"])
print("action_weights:", state["A001"]["action_weights"])


Baseline CS: [{'Country': 'AT', 'solution': 'MRS', 'next_action': 'Email', 'count': 204}, {'Country': 'AT', 'solution': 'MRS', 'next_action': 'Follow Up', 'count': 2}, {'Country': 'AT', 'solution': 'MRS', 'next_action': '1St Appointment', 'count': 1}, {'Country': 'AT', 'solution': 'MRS', 'next_action': '2Nd Appointment', 'count': 1}]
After Email: 0.050000000000000044
[{'Country': 'AT', 'solution': 'MRS', 'next_action': 'Email', 'count': 204, 'base_weight': 1.0, 'account_weight': 0.050000000000000044, 'repeat_penalty': 0.7, 'score': 7.140000000000007}, {'Country': 'AT', 'solution': 'MRS', 'next_action': 'Follow Up', 'count': 2, 'base_weight': 1.0, 'account_weight': 0.95, 'repeat_penalty': 1.0, 'score': 1.9}, {'Country': 'AT', 'solution': 'MRS', 'next_action': '1St Appointment', 'count': 1, 'base_weight': 1.0, 'account_weight': 1.0, 'repeat_penalty': 1.0, 'score': 1.0}, {'Country': 'AT', 'solution': 'MRS', 'next_action': '2Nd Appointment', 'count': 1, 'base_weight': 1.0, 'account_weight'